# Parameter efficient finetuning of llama2-7b on labeled Enron email corpus
This notebook runs on a T4 GPU.

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    

In [2]:
import pandas as pd
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"

# The instruction dataset to use
dataset_name = "neelblabla/enron_labeled_unprocessed_email-llama2-7b_finetuning"

# Fine-tuned model name
new_model = "email-categorization-llama2-7b-peft"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [4]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="prompts",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.169000
50,0.560600
75,1.437000
100,0.459200
125,1.403700
150,0.443700
175,1.351100
200,0.431600
225,1.477500
250,0.474700


In [5]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [6]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our model
prompt_mail_subject = "Transredes spill"
prompt_mail_body = "Do we have someone who works for us now who was at Transredes when the government owned it and could help us do a comparison between our respective environmental records? It seems to me that even though we are having our share of problems, we have probably had fewer problems than the government did and have probably been more responsive and thorough in our clean up efforts. Obviously, we would need to be judicious about how we used such information (we don't want to aggravate the government), but having such an analysis may be helpful in showing the progress made and in discouraging overly punitive action by the government. What do you think?"
# prompt_mail_category = 'Company Business/Strategy'
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
categorization_prompt = (
    "You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n"
    "1. 'Company Business/Strategy'\n"
    "2. 'Purely Personal'\n"
    "3. 'Personal but in a professional context'\n"
    "4. 'Logistic Arrangements'\n"
    "5. 'Employment arrangements'\n"
    "6. 'Document editing/checking/collaboration'\n"
    "7. 'Empty message (due to missing attachment)'\n"
    "8. 'Empty message'\n"
    "Please provide only one category (e.g., 'Purely Personal').\n"
    "\n"
    "Subject:: {prompt_mail_subject}\n"
    "Body:: {prompt_mail_body}\n"
    "\n"
    "What should this email be categorized as?\n"
)
result = pipe(categorization_prompt.format(prompt_mail_subject=prompt_mail_subject, prompt_mail_body=prompt_mail_body)) # generating text using the modified prompt
print(result[0]['generated_text'])



/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:
1. 'Company Business/Strategy'
2. 'Purely Personal'
3. 'Personal but in a professional context'
4. 'Logistic Arrangements'
5. 'Employment arrangements'
6. 'Document editing/checking/collaboration'
7. 'Empty message (due to missing attachment)'
8. 'Empty message'
Please provide only one category (e.g., 'Purely Personal').

Subject:: Transredes spill
Body:: Do we have someone who works for us now who was at Transredes when the government owned it and could help us do a comparison between our respective environmental records? It seems to me that even though we are having our share of problems, we have probably had fewer problems than the government did and have probably been more responsive and thorough in our clean up efforts. Obviously, we would need to be judicious about how we used such information (we don't want to aggrav

In [7]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

19965

In [8]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our model
prompt_mail_subject = "Transredes spill"
prompt_mail_body = "Do we have someone who works for us now who was at Transredes when the government owned it and could help us do a comparison between our respective environmental records? It seems to me that even though we are having our share of problems, we have probably had fewer problems than the government did and have probably been more responsive and thorough in our clean up efforts. Obviously, we would need to be judicious about how we used such information (we don't want to aggravate the government), but having such an analysis may be helpful in showing the progress made and in discouraging overly punitive action by the government. What do you think?"
# prompt_mail_category = 'Company Business/Strategy'
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
categorization_prompt = (
    "You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n"
    "1. 'Company Business/Strategy'\n"
    "2. 'Purely Personal'\n"
    "3. 'Personal but in a professional context'\n"
    "4. 'Logistic Arrangements'\n"
    "5. 'Employment arrangements'\n"
    "6. 'Document editing/checking/collaboration'\n"
    "7. 'Empty message (due to missing attachment)'\n"
    "8. 'Empty message'\n"
    "Please provide only one category (e.g., 'Purely Personal').\n"
    "\n"
    "Subject:: {prompt_mail_subject}\n"
    "Body:: {prompt_mail_body}\n"
    "\n"
    "What should this email be categorized as?\n"
)
result = pipe(categorization_prompt.format(prompt_mail_subject=prompt_mail_subject, prompt_mail_body=prompt_mail_body)) # generating text using the modified prompt
print(result[0]['generated_text'])

You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:
1. 'Company Business/Strategy'
2. 'Purely Personal'
3. 'Personal but in a professional context'
4. 'Logistic Arrangements'
5. 'Employment arrangements'
6. 'Document editing/checking/collaboration'
7. 'Empty message (due to missing attachment)'
8. 'Empty message'
Please provide only one category (e.g., 'Purely Personal').

Subject:: Transredes spill
Body:: Do we have someone who works for us now who was at Transredes when the government owned it and could help us do a comparison between our respective environmental records? It seems to me that even though we are having our share of problems, we have probably had fewer problems than the government did and have probably been more responsive and thorough in our clean up efforts. Obviously, we would need to be judicious about how we used such information (we don't want to aggrav

In [10]:
import pandas as pd
# Generating responses of fine-tuned model on test dataset for further analysis

# IMPORTANT: Check the data on the URL if it is in the required format. Else, first host the data in the correct format on this URL.
test_df_csv_url = "https://raw.githubusercontent.com/neelblabla/large_language_models_for_processing_emails/main/1_finetuning_LLAMA2-7b/1a_data_preparation_and_eda/prompts_test.csv"
test_df = pd.read_csv(test_df_csv_url)

# We faced computation memory challenge and thus limited prompt-lengths to the below number
max_prompt_length = 300
filtered_test_df = test_df[test_df['input_test_prompts'].apply(lambda x: len(x.split()) <= max_prompt_length)]
len(filtered_test_df)


129

In [11]:
# Generating model outputs by iterating over prompts and storing them in a new column in the dataframe
model_outputs = []
for index, row in filtered_test_df.iterrows():
    input_prompt = row['input_test_prompts']
    input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=350, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    model_outputs.append(generated_text)

filtered_test_df['model_output'] = model_outputs

# Save the DataFrame to a CSV file
filtered_test_df.to_csv('model_test_output.csv', index=False)

# Download the CSV file to your local machine
from google.colab import files
files.download('model_test_output.csv')

<ipython-input-11-d69d80af2d44>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df['model_output'] = model_outputs


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# Finally, we push merged model to Hugging Face repo for future inferencing

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/neelblabla/email-categorization-llama2-7b-peft/commit/f82c296818062262412935ef6583c8ef84514795', commit_message='Upload tokenizer', commit_description='', oid='f82c296818062262412935ef6583c8ef84514795', pr_url=None, pr_revision=None, pr_num=None)